In [15]:
import numpy as np

# Define simulation function
def simulate_boarding(method, disobedience_coefficient):

    # Initialize
    # Define the layout of the flying wing aircraft
    n_rows = 11  # Rows in the first and fifth blocks
    n_rows_middle = 14  # Rows in the second, third, and fourth blocks
    n_cols = 6  # Seats per row

    # Calculate the total number of passengers
    n_pass = 2 * n_rows + 3 * n_rows_middle

    # Create seat matrix, using -1 to represent unoccupied seats
    seats = np.zeros((n_rows * 2 + n_rows_middle * 3, n_cols))
    seats[:, :] = -1

    # Create aisle array
    aisle_q = np.zeros(n_rows * 2 + n_rows_middle * 3)
    aisle_q[:] = -1

    # Create initial passenger number queue
    pass_q = [int(i) for i in range(n_pass)]
    pass_q = np.array(pass_q)

    # Account for passenger ages and physical passenger disabilities
    # Define the parameters for the age distribution
    mean_age = 40
    stddev_age = 15

    # Generate passenger ages based on the custom distribution
    passenger_ages = np.random.normal(mean_age, stddev_age, n_pass)
    # Ensure the ages are non-negative
    passenger_ages = np.maximum(passenger_ages, 0)
    # Round the ages to whole numbers
    passenger_ages = np.round(passenger_ages)

    passenger_disabilities = np.random.choice([0, 1], n_pass, p=[0.95, 0.05])  # 5% chance of having a disability

    # Create array for seat numbers
    row_q_init = np.zeros(n_pass)
    col_q_init = np.zeros(n_pass)

    # Let's create moveto arrays
    moveto_loc = np.zeros(n_pass)
    moveto_time = np.zeros(n_pass)

    moveto_loc_dict = {i: j for i in pass_q for j in moveto_loc}
    moveto_time_dict = {i: j for i in pass_q for j in moveto_time}

    # Create function to assign seat number to each passenger
    def AssignSeats(rq, cq, assign_type, n_pass=n_pass, n_rows=n_rows, n_rows_middle=n_rows_middle):
        if assign_type == "SINP":
            # Initialize initial and final positions
            i = 0
            f = n_rows * 2  # Rows in the first and fifth blocks

            # Define column seating positions
            c = [0, 5, 1, 4, 2, 3]

            # Define iteration counter
            count = 0

            # Assign queue
            while f <= n_pass:
                if count < 2:
                    rq[i:f] = list(reversed(range(0, n_rows * 2)))
                else:
                    rq[i:f] = list(reversed(range(0, n_rows_middle * 3)))
                cq[i:f] = [c[count % 6]] * (f - i)
                i += (f - i)
                f += (n_rows * 2) if count < 2 else (n_rows_middle * 3)
                count += 1

        if assign_type == "Random":
            av_rows = np.arange(0, n_rows * 2 if n_rows * 2 < n_pass else n_pass, 1)
            av_rows = np.tile(av_rows, (n_cols, 1))
            av_rows = av_rows.T.flatten()

            av_cols = np.arange(0, n_cols, 1)
            av_cols = np.tile(av_cols, (n_rows * 2 if n_rows * 2 < n_pass else n_pass, 1)).flatten()

            av_seats = np.zeros((n_pass, 2))
            for i in range(n_pass):
                av_seats[i] = [av_rows[i], av_cols[i]]

            np.random.shuffle(av_seats)
            rq = av_seats[:, 0]
            cq = av_seats[:, 1]

        if assign_type == "BTF":
            av_rows = np.arange(0, n_rows_middle * 3, 1)
            av_rows = np.tile(av_rows, (n_cols, 1))
            av_rows = av_rows.T.flatten()
            av_cols = np.arange(0, n_cols, 1)
            av_cols = np.tile(av_cols, (n_rows_middle * 3, 1)).flatten()
            av_seats = np.zeros((n_pass, 2))
            for i in range(n_pass):
                av_seats[i] = [av_rows[i], av_cols[i]]

            group1 = av_seats[:n_rows_middle]
            np.random.shuffle(group1)
            group2 = av_seats[n_rows_middle:2 * n_rows_middle]
            np.random.shuffle(group2)
            group3 = av_seats[2 * n_rows_middle:]
            np.random.shuffle(group3)
            av_seats_final = np.concatenate((group3, group2, group1))
            rq = av_seats_final[:, 0]
            cq = av_seats_final[:, 1]

        if assign_type == "FTB":
            av_rows = np.arange(0, n_rows_middle * 3, 1)
            av_rows = np.tile(av_rows, (n_cols, 1))
            av_rows = av_rows.T.flatten()
            av_cols = np.arange(0, n_cols, 1)
            av_cols = np.tile(av_cols, (n_rows_middle * 3, 1)).flatten()
            av_seats = np.zeros((n_pass, 2))
            for i in range(n_pass):
                av_seats[i] = [av_rows[i], av_cols[i]]

            group1 = av_seats[:n_rows_middle]
            np.random.shuffle(group1)
            group2 = av_seats[n_rows_middle:2 * n_rows_middle]
            np.random.shuffle(group2)
            group3 = av_seats[2 * n_rows_middle:]
            np.random.shuffle(group3)
            av_seats_final = np.concatenate((group1, group2, group3))
            rq = av_seats_final[:, 0]
            cq = av_seats_final[:, 1]

        if assign_type == "WMA":
            window_rows = np.tile([0, 5, 0, 5], n_rows_middle)
            _rows = np.tile([1, 4, 1, 4], n_rows_middle)
            middle_rows = np.tile([2, 3, 2, 3], n_rows_middle)
            av_rows = np.concatenate((window_rows, middle_rows))
            av_cols = np.tile(np.arange(n_cols), 2 * n_rows_middle)
            av_seats = np.zeros((n_pass, 2))
            for i in range(n_pass):
                av_seats[i] = [av_rows[i], av_cols[i]]
            rq = av_seats[:, 0]
            cq = av_seats[:, 1]

        return rq, cq

    # Create function to move passengers into the aircraft
    def MoveToAisle(t, aisle_q, pass_q, sum_time):
        if t > sum_time[0]:
            if aisle_q[0] == -1:
                aisle_q[0] = pass_q[0].copy()
                pass_q = np.delete(pass_q, 0)
                sum_time = np.delete(sum_time, 0)
        return aisle_q, pass_q, sum_time

    def get_walk_speed(age, disability):
        if age > 70 or age < 8:
            if disability == 1:
                mean_time = 1.2  # 20% increase for age > 70 or < 8 and disability
            else:
                mean_time = 1.2  # 20% increase for age > 70 or < 8 and no disability
        elif disability == 1:
            mean_time = 1.5  # 50% increase for passengers with disabilities
        else:
            mean_time = 1.0  # Default mean_time for other passengers

        stddev_time = 0.2
        walk_speed = np.random.normal(loc=mean_time, scale=stddev_time)
        return walk_speed

    # Create an array of walking speeds based on age and disability
    walk_speeds = [get_walk_speed(age, disability) for age, disability in zip(passenger_ages, passenger_disabilities)]

    # Calculate total boarding time based on the chosen method
    time = 0
    sum_time = [0]
    aisle_time = 0

    rq, cq = AssignSeats(row_q_init, col_q_init, method)
    total_pass = n_pass
    aisle_q, pass_q, sum_time = MoveToAisle(0, aisle_q, pass_q, sum_time)
    time = time + aisle_time

    while total_pass > 0:
        for i in range(len(aisle_q)):
            if aisle_q[i] >= 0:
                if aisle_q[i] == time:
                    aisle_q[i] = -1
                    aisle_q, pass_q, sum_time = MoveToAisle(time, aisle_q, pass_q, sum_time)
            else:
                aisle_q[i] = aisle_q[i] - walk_speeds[i] / 60  # Updated to use `i` directly
        total_pass = len(pass_q)
        if total_pass == 0:
            break
        time = time + 1

    return time




In [ ]:
# Number of simulations
num_simulations = 1


boarding_times = []

for _ in range(num_simulations):
    result = simulate_boarding("Random", 0.05)
    if result > 0:
        boarding_times.append(result)
    else:
        print("negative number!")

# Calculate the average time
average_time = sum(boarding_times) / len(boarding_times)

# Calculate percentiles
percentile_5 = np.percentile(boarding_times, 5)
percentile_95 = np.percentile(boarding_times, 95)

print("Average boarding time for {} simulations: {:.2f}".format(len(boarding_times), average_time))
print("5th percentile boarding time: {:.2f}".format(percentile_5))
print("95th percentile boarding time: {:.2f}".format(percentile_95))

# Create a histogram of boarding times
plt.hist(boarding_times, bins=20, color='blue', alpha=0.7)
plt.xlabel("Boarding Time")
plt.ylabel("Frequency")
plt.title("Histogram of Boarding Times")
plt.grid(True)
plt.show()